In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark

In [5]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/Stream/OneDrive_2024-10-22.zip'  # Change this to your ZIP file path
extract_path = '/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/Stream/'  # Path where you want to extract the files

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from time import sleep

In [4]:
spark = SparkSession.builder.appName('Basic streaming').getOrCreate()

In [7]:
#Get schema of dataset
static = spark.read.json("/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/Stream/Streaming Dataset/activity-data")
static.printSchema()
dataSchema = static.schema

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [8]:
# Create Streming Dataset
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
  .json("/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/Stream/Streaming Dataset/activity-data")

In [9]:
# Get required info from dataset
activityCounts = streaming.groupBy("gt").count()

In [10]:
# Specify the numner of shuffle partitions
spark.conf.set("spark.sql.shuffle.partitions", 1)

In [11]:
# Generate query by specifying output sink using writeStream
activityQuery = activityCounts.writeStream.queryName("activity_count")\
  .format("memory").outputMode("complete")\
  .start()

In [12]:
# Any number of queries (usecases) can be created
#userCounts = streaming.groupBy("").count()
userCounts = streaming.groupBy('User').count()
userQuery = userCounts.writeStream.queryName("user_count")\
  .format("memory").outputMode("complete")\
  .start()

In [13]:
# Display result
for x in range(5):
    spark.sql('SELECT * FROM activity_count').show()
    spark.sql('SELECT * FROM user_count').show()
    sleep(10)

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+----+-----+
|User|count|
+----+-----+
+----+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|52260|
|     stand|56924|
|       sit|61547|
|      walk|66280|
|      bike|53984|
|      null|52239|
|stairsdown|46825|
+----------+-----+

+----+-----+
|User|count|
+----+-----+
|   h|38665|
|   e|48015|
|   d|40620|
|   c|38575|
|   g|45839|
|   i|46275|
|   f|46030|
|   b|45615|
|   a|40425|
+----+-----+

+----------+------+
|        gt| count|
+----------+------+
|  stairsup|125431|
|     stand|136619|
|       sit|147699|
|      walk|159072|
|      bike|129572|
|      null|125379|
|stairsdown|112371|
+----------+------+

+----+------+
|User| count|
+----+------+
|   h| 92796|
|   e|115236|
|   d| 97488|
|   c| 92580|
|   g|110015|
|   i|111060|
|   f|110472|
|   b|109476|
|   a| 97020|
+----+------+

+----------+------+
|        gt| count|
+----------+------+
|  stairsup|156800|
|     stand|170778|
|       sit|18

In [14]:
# Run nonstop
activityQuery.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [15]:
import sys
print(sys.version)

3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


In [16]:
!python --version

Python 3.10.12


In [17]:
sys.version

'3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]'

In [18]:
!python --version

Python 3.10.12
